# 딥

===== OS / Python 환경 =====

OS: Linux 6.8.0-1032-oracle #33~22.04.1-Ubuntu SMP Thu Aug 14 01:46:51 UTC 2025

Machine: x86_64

Processor: x86_64

Python: 3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 20:50:58) [GCC 12.3.0]

In [9]:
# =========================================================
# Final Round — Robust, GPU-ready, Error-proof
# LSTM + Lightweight N-HiTS (two-head) with full feature pipeline
# - Paths: use exactly the competitionㅁ layout you provided
# - New metric: (A + B + C + D) / 4 maximized
# - No KeyError on '영업장명_메뉴명' (header/key sanitization)
# - No NaN/inf in log1p
# - No interop thread errors (no set_num_interop_threads)
# =========================================================

# ----------------------------- Setup -----------------------------
import os, re, glob, random, json
SEED = 42

os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")
os.environ.setdefault("CUBLAS_WORKSPACE_CONFIG", ":16:8")

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Holidays (fallback if not installed)
try:
    import holidays
    HAVE_HOLIDAYS = True
except Exception:
    HAVE_HOLIDAYS = False

# RNG/Determinism
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)
torch.use_deterministic_algorithms(True, warn_only=False)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False
torch.set_float32_matmul_precision("high")  # ok on CPU/GPU

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", DEVICE)

# -------------------------- Paths & HPs --------------------------
LOOKBACK, HORIZON = 28, 7
BATCH, EPOCHS, LR = 256, 25, 1e-3
PATIENCE = 5

# ---- Use the exact paths you provided ----
DATA_PATH = "../data"
PRICE_PATH = DATA_PATH+"/train/price.csv"
ROOM_TYPE_PATH = DATA_PATH+"/train/room_type.csv"
TRAIN_PATH = DATA_PATH+"/train/train.csv"
TRAIN_group_PATH   = DATA_PATH+"/train/meta/TRAIN_group.csv"
TRAIN_hwadam_PATH  = DATA_PATH+"/train/meta/TRAIN_hwadam.csv"
TRAIN_room_PATH    = DATA_PATH+"/train/meta/TRAIN_room.csv"
TRAIN_ski_PATH     = DATA_PATH+"/train/meta/TRAIN_ski.csv"
TRAIN_weather_PATH = DATA_PATH+"/train/meta/TRAIN_weather.csv"
TEST_DIR      = DATA_PATH+"/test"     # TEST_00.csv ~ TEST_09.csv
TEST_meta_DIR = DATA_PATH+"/test/meta"
SAMPLE_PATH   = DATA_PATH+"/sample_submission.csv"
SUBMIT_PATH   = DATA_PATH+"/submission.csv"

# Blending/temperature
GROUPING = 'store'        # 'store' | 'category' | 'store_category'
MIN_ITEMS_PER_GROUP = 3
GLOBAL_ALPHA = 0.80
ALPHAS_GRID = np.arange(0.40, 1.01, 0.01)
TEMP_GRID   = [0.5, 0.75, 1.0, 1.25, 1.5]

# ---------------------- Safe torch.load helper -------------------
def _load_state_dict_safe(path, map_location=None):
    # Avoids future pickle warning (PyTorch 2.5+). Try weights_only.
    try:
        return torch.load(path, map_location=map_location, weights_only=True)
    except TypeError:
        return torch.load(path, map_location=map_location)

# ---------------------- Column/Key Sanitizers --------------------
CANON_KEY = "영업장명_메뉴명"

def sanitize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    clean = []
    for c in df.columns:
        s = str(c)
        s = s.replace("\ufeff","").replace("\u200b","").replace("\xa0"," ")
        s = s.strip()
        clean.append(s)
    df.columns = clean
    return df

def sanitize_key_values(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if CANON_KEY in df.columns:
        df[CANON_KEY] = (df[CANON_KEY].astype(str)
                         .str.replace("\ufeff|\u200b", "", regex=True)
                         .str.replace("\xa0", " ", regex=False)
                         .str.replace(r"\s+", " ", regex=True)
                         .str.strip())
    return df

def normalize_key_column(df: pd.DataFrame) -> pd.DataFrame:
    df = sanitize_columns(df)
    df = df.copy()
    if CANON_KEY in df.columns:
        return sanitize_key_values(df)

    cand = [c for c in df.columns
            if ('메뉴' in c) and (('영업장' in c) or ('매장' in c) or ('업장' in c) or ('지점' in c))]
    if len(cand) == 1:
        df = df.rename(columns={cand[0]: CANON_KEY})
        return sanitize_key_values(df)

    def _mk(sm_col, mn_col):
        return (df[sm_col].astype(str).str.strip() + "_" + df[mn_col].astype(str).str.strip())

    for s_col in ['영업장명','매장명','업장명','지점명']:
        if s_col in df.columns and '메뉴명' in df.columns:
            df[CANON_KEY] = _mk(s_col, '메뉴명')
            return sanitize_key_values(df)

    raise KeyError(f"'{CANON_KEY}' 컬럼을 찾거나 만들 수 없습니다. 현재 컬럼: {list(df.columns)}")

# ---------------------- IO helpers & features --------------------
def read_csv_safe(path, **kwargs):
    for enc in [None, "utf-8-sig", "cp949", "euc-kr"]:
        try:
            return pd.read_csv(path, encoding=enc, **kwargs)
        except Exception:
            pass
    raise RuntimeError(f"파일 로드 실패: {path}")

def preprocess_price(price_df: pd.DataFrame) -> pd.DataFrame:
    key_col, price_col = CANON_KEY, "평균판매금액"
    df = normalize_key_column(price_df)
    df[price_col] = pd.to_numeric(df[price_col], errors="coerce")
    miss0 = df[price_col].isna() | (df[price_col] == 0)

    # 담하 (정식) 보정
    is_damha   = df[key_col].str.contains("담하", na=False)
    is_jungsik = df[key_col].str.contains(r"\(정식\)", na=False)
    damha_mean = df.loc[is_damha & is_jungsik & ~miss0, price_col].mean()
    if np.isnan(damha_mean):
        damha_mean = df.loc[is_damha & ~miss0, price_col].mean()
    df.loc[is_damha & is_jungsik & miss0, price_col] = damha_mean

    # 고정값
    fixed = {"미라시아_브런치 2인 패키지": 81000, "미라시아_브런치 4인 패키지": 162000}
    for k, v in fixed.items():
        df.loc[(df[key_col] == k) & miss0, price_col] = v

    # 오픈푸드 → 0
    openfood = df[key_col].str.contains("Open Food|오픈푸드", na=False)
    df.loc[openfood & df[price_col].isna(), price_col] = 0
    return df[[key_col, price_col]].drop_duplicates(subset=[key_col], keep="last")

# 객실 타입 메타
room_type_raw = read_csv_safe(ROOM_TYPE_PATH)
room_type_raw = sanitize_columns(room_type_raw)
room_type = room_type_raw.rename(columns={"객실타입": "room_code"})
room_type_meta = room_type[['room_code', '객실타입명']].copy()

def build_room_ratio(room_csv_path: str, room_type_meta: pd.DataFrame) -> pd.DataFrame:
    room_wide = read_csv_safe(room_csv_path)
    room_wide = sanitize_columns(room_wide)
    room_wide["영업일자"] = pd.to_datetime(room_wide["영업일자"])
    room_long = room_wide.melt(id_vars=["영업일자"], var_name="room_code", value_name="room_count")
    room_long["room_count"] = pd.to_numeric(room_long["room_count"], errors="coerce").fillna(0)
    room_long = room_long.merge(room_type_meta, on="room_code", how="left")
    room_long["취사여부"] = room_long["객실타입명"].fillna("").str.contains("취사").astype(int)
    room_long["cook_cnt"] = room_long["취사여부"] * room_long["room_count"]

    daily = room_long.groupby("영업일자", as_index=False).agg(
        total_rooms   = ("room_count","sum"),
        cooking_rooms = ("cook_cnt","sum")
    )
    daily["cooking_ratio"] = np.where(
        daily["total_rooms"] > 0,
        daily["cooking_rooms"] / daily["total_rooms"],
        np.nan
    )
    return daily[["영업일자", "cooking_ratio"]]

def attach_ratio(df: pd.DataFrame, ratio_df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["영업일자"] = pd.to_datetime(out["영업일자"])
    ratio_df = ratio_df.copy()
    ratio_df["영업일자"] = pd.to_datetime(ratio_df["영업일자"])
    return out.merge(ratio_df, on="영업일자", how="left")

def _coerce_ski_colnames(sdf: pd.DataFrame) -> pd.DataFrame:
    sdf = sanitize_columns(sdf)
    cols = {c: c.replace(" ", "") for c in sdf.columns}
    sdf = sdf.rename(columns=cols)
    if "1일내장객" not in sdf.columns:
        raise KeyError("스키 데이터에 '1일내장객' 컬럼이 없습니다.")
    return sdf

def attach_daily_ski(df: pd.DataFrame, sdf: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["영업일자"]  = pd.to_datetime(out["영업일자"], errors="coerce")
    sdf = _coerce_ski_colnames(sdf)
    sdf["영업일자"] = pd.to_datetime(sdf["영업일자"], errors="coerce")
    sdf["1일내장객"] = pd.to_numeric(sdf["1일내장객"], errors="coerce")
    daily = sdf.groupby("영업일자", as_index=False)["1일내장객"].max()
    return out.merge(daily, on="영업일자", how="left").fillna({"1일내장객":0})

def add_time_features(df, date_col='영업일자'):
    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col])
    dt = out[date_col].dt
    month = dt.month
    out['일']   = dt.day
    out['요일'] = dt.dayofweek
    out['주차'] = dt.isocalendar().week.astype('int32')
    out['계절'] = month.map(lambda m: 0 if m in [12,1,2] else 1 if m in [3,4,5] else 2 if m in [6,7,8] else 3)
    # Holidays (safe fallback to empty)
    if HAVE_HOLIDAYS:
        years = sorted(out[date_col].dt.year.unique())
        kr_holidays = set(holidays.KR(years=years).keys())
    else:
        kr_holidays = set()
    out['공휴일여부'] = out[date_col].dt.date.isin(kr_holidays).astype(int)
    out['월_sin']   = np.sin(2*np.pi*month/12)
    out['월_cos']   = np.cos(2*np.pi*month/12)
    out['요일_sin'] = np.sin(2*np.pi*out['요일']/7)
    out['요일_cos'] = np.cos(2*np.pi*out['요일']/7)
    return out

def add_calendar_context_features(df: pd.DataFrame, date_col='영업일자') -> pd.DataFrame:
    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col])
    d = out[date_col].dt
    if 'is_weekend' not in out.columns:
        out['is_weekend'] = (d.dayofweek >= 5).astype(int)
    if HAVE_HOLIDAYS:
        years = sorted(out[date_col].dt.year.unique())
        kr_hdays = set(holidays.KR(years=years).keys())
    else:
        kr_hdays = set()
    prev_day = (out[date_col] - pd.Timedelta(days=1)).dt.date
    next_day = (out[date_col] + pd.Timedelta(days=1)).dt.date
    out['전일_공휴일'] = prev_day.map(lambda x: int(x in kr_hdays))
    out['익일_공휴일'] = next_day.map(lambda x: int(x in kr_hdays))
    if '공휴일여부' in out.columns:
        out['휴일전날']   = ((out['공휴일여부'] == 0) & (out['익일_공휴일'] == 1)).astype(int)
        out['휴일다음날'] = ((out['공휴일여부'] == 0) & (out['전일_공휴일'] == 1)).astype(int)
        out['연휴중여부'] = (((out['전일_공휴일'] == 1) | (out['익일_공휴일'] == 1)) & (out['공휴일여부'] == 1)).astype(int)
    else:
        out['휴일전날']   = (out['익일_공휴일'] == 1).astype(int)
        out['휴일다음날'] = (out['전일_공휴일'] == 1).astype(int)
        out['연휴중여부'] = ((out['전일_공휴일'] == 1) | (out['익일_공휴일'] == 1)).astype(int)
    return out

def add_filtered_ts_features(
    df: pd.DataFrame,
    group_cols=(CANON_KEY,),
    date_col='영업일자',
    target_col='매출수량',
    lags=(7, 14),
    roll_mean_windows=(1, 3, 7, 14, 28),
    roll_std_windows=(3, 7, 14, 28),
):
    out = df.sort_values(list(group_cols)+[date_col]).copy()
    g = out.groupby(list(group_cols))[target_col]
    prev = g.shift(1)
    keys = [out[c] for c in group_cols]

    prev_exp_mean = (
        prev.groupby(keys)
            .expanding(min_periods=1).mean()
            .reset_index(level=list(range(len(group_cols))), drop=True)
    ).fillna(0.0)

    for k in lags:
        out[f'lag_{k}'] = g.shift(k).fillna(prev_exp_mean)

    for w in roll_mean_windows:
        out[f'roll_mean_{w}'] = (
            prev.groupby(keys).rolling(w, min_periods=1).mean()
                .reset_index(level=list(range(len(group_cols))), drop=True)
        ).fillna(0.0)

    for w in roll_std_windows:
        out[f'roll_std_{w}'] = (
            prev.groupby(keys).rolling(w, min_periods=2).std()
                .reset_index(level=list(range(len(group_cols))), drop=True)
        ).fillna(0.0)

    return out

def finalize_features(df: pd.DataFrame) -> pd.DataFrame:
    key = CANON_KEY
    out = add_time_features(df, date_col='영업일자')
    out = add_calendar_context_features(out, date_col='영업일자')
    out = out.sort_values([key, '영업일자']).copy()

    # wd_mean_local_28 (apply 없이 — 키 증발 방지)
    s = pd.to_numeric(out['매출수량'], errors='coerce').fillna(0.0)
    same_wd_mean = (
        s.groupby(out[key]).shift(7)
         .groupby(out[key]).rolling(4, min_periods=1).mean()
         .reset_index(level=0, drop=True)
    )
    all_wd_mean = (
        s.groupby(out[key]).shift(1)
         .groupby(out[key]).rolling(7, min_periods=1).mean()
         .reset_index(level=0, drop=True)
    )
    out['wd_mean_local_28'] = same_wd_mean.where(~same_wd_mean.isna(), all_wd_mean).fillna(0.0).astype(float)

    out = add_filtered_ts_features(out, group_cols=(key,),
                                   date_col='영업일자', target_col='매출수량')
    return out

def add_venue_group_features(df: pd.DataFrame,
                             col=CANON_KEY,
                             keep_venue_name=True) -> pd.DataFrame:
    out = df.copy()
    def _extract_venue(x: str) -> str:
        return str(x).split("_",1)[0] if pd.notna(x) else ""
    venue = out[col].astype(str).map(_extract_venue)
    if keep_venue_name:
        out['영업장명'] = venue

    ski_venues     = {"카페테리아","포레스트릿"}
    hwadam_venues  = {"화담숲주막","화담숲카페"}
    lodging_venues = {"느티나무 셀프BBQ","연회장","담하","미라시아","라그로타"}
    def _map_group(v):
        if v in ski_venues: return "ski"
        if v in hwadam_venues: return "hwadam"
        if v in lodging_venues: return "lodging"
        return "other"
    group_name = venue.map(_map_group)
    out['venue_group_label'] = group_name.map({"ski":0,"hwadam":1,"lodging":2,"other":-1}).astype('int8')

    cats = pd.Categorical(group_name, categories=["ski","hwadam","lodging"])
    dummies = pd.get_dummies(cats, prefix='grp')
    for c in ['grp_ski','grp_hwadam','grp_lodging']:
        if c not in dummies: dummies[c] = 0
    return pd.concat([out, dummies[['grp_ski','grp_hwadam','grp_lodging']].astype('int8')], axis=1)

# ----------------------- Build features & master df -------------------------
def build_feature_frames_in_memory_all():
    key_col, target = CANON_KEY, "매출수량"
    price_raw  = read_csv_safe(PRICE_PATH)
    price_slim = preprocess_price(price_raw)

    # Train
    train = read_csv_safe(TRAIN_PATH, parse_dates=["영업일자"])
    train = normalize_key_column(train)
    assert key_col in train.columns, f"키 없음: {list(train.columns)}"
    train[target] = pd.to_numeric(train[target], errors='coerce').fillna(0).clip(lower=0)
    train = train.merge(price_slim, on=key_col, how="left")
    train_ratio = build_room_ratio(TRAIN_room_PATH, room_type_meta)
    train = attach_ratio(train, train_ratio)
    train_ski  = read_csv_safe(TRAIN_ski_PATH)
    train = attach_daily_ski(train, train_ski)
    train = train.sort_values([key_col, "영업일자"])
    train = finalize_features(train)
    train_X = add_venue_group_features(train.drop(columns=[target], errors="ignore"), col=key_col, keep_venue_name=True)
    train_y = train[target].copy()

    # Test
    test_X_dict = {}
    for tf in sorted(glob.glob(os.path.join(TEST_DIR, "TEST_*.csv"))):
        sid = Path(tf).stem.split("_")[-1]
        tdf = read_csv_safe(tf, parse_dates=["영업일자"])
        tdf = normalize_key_column(tdf)
        assert key_col in tdf.columns, f"{tf} 키 없음: {list(tdf.columns)}"
        tdf[target] = pd.to_numeric(tdf.get(target, 0), errors='coerce').fillna(0).clip(lower=0)
        tdf = tdf.merge(price_slim, on=key_col, how="left")
        trot = build_room_ratio(os.path.join(TEST_meta_DIR, f"TEST_room_{sid}.csv"), room_type_meta)
        tdf  = attach_ratio(tdf, trot)
        tsdf = read_csv_safe(os.path.join(TEST_meta_DIR, f"TEST_ski_{sid}.csv"))
        tdf  = attach_daily_ski(tdf, tsdf)
        tdf  = tdf.sort_values([key_col, "영업일자"])
        tdf  = finalize_features(tdf)
        test_X = add_venue_group_features(tdf.drop(columns=[target], errors="ignore"), col=key_col, keep_venue_name=True)
        test_X_dict[f"TEST_{sid}"] = test_X

    # Align test columns to train_X
    base_cols = list(train_X.columns)
    for k, df in test_X_dict.items():
        for c in base_cols:
            if c not in df.columns:
                df[c] = 0 if c.startswith("grp_") else np.nan
        test_X_dict[k] = df[base_cols]

    return train_X, train_y, test_X_dict

train_X, train_y, testX_dict = build_feature_frames_in_memory_all()

# Make master train df (with target)
train_df = train_X.copy()
train_df["매출수량"] = pd.to_numeric(train_y, errors='coerce').fillna(0).clip(lower=0)

# --------------- Classic helpers for store/menu/category --------------------
def split_store_menu(df):
    df = df.copy()
    if ('영업장명' in df.columns) and ('메뉴명' in df.columns):
        return df
    df = normalize_key_column(df)
    sp = df[CANON_KEY].astype(str).str.split('_', n=1, expand=True)
    df['영업장명'] = sp[0].str.strip()
    df['메뉴명']  = sp[1].str.strip() if sp.shape[1] > 1 else ""
    return df

def categorize_menu(menu_name: str) -> str:
    name = str(menu_name)
    if any(k in name for k in ['연회장','Conference','Grand Ballroom','Convention','OPUS','Hall','(단체)','무제한','단체식']): return '행사/연회'
    if any(k in name for k in ['대여','대여료','렌탈','잔디그늘집','의자','룸 이용료','룸']): return '대여'
    if any(k in name for k in ['막걸리','소주','참이슬','처음처럼','카스','테라','하이네켄','버드와이저','스텔라','와인','글라스와인','Gls','Beer','맥주','하이볼','칵테일','복분자']): return '주류'
    if any(k in name for k in ['콜라','코카콜라','제로','스프라이트','에이드','아메리카노','라떼','아이스티','생수','커피','식혜','미숫가루','메밀미숫가루']): return '음료'
    if any(k in name for k in ['공깃밥','주먹밥','야채추가','빵 추가','면 사리','파스타면 추가','쌈야채','친환경 접시','접시']): return '사이드'
    if any(k in name for k in ['패키지','Platter','세트','Open Food','오픈푸드','브런치']): return '세트/패키지'
    if any(k in name for k in ['비빔밥','찌개','국밥','탕','전골','돈까스','리조또','파스타','스파게티','떡볶이','우동','짬뽕','짜장','삼겹','양갈비','샐러드','핫도그','치즈','우거지','된장','냉면','물냉면','비빔냉면','김치찌개','떡갈비','불고기','갈비탕','랍스타','스튜','AUS','한우']): return '메인'
    if any(k in name for k in ['수저','젓가락','컵','종이컵','소주컵','일회용','접시','쌈장','허브솔트','햇반','라면사리','샷 추가']): return '소모품'
    return '기타'

def make_zero_streak(x):
    z = (np.asarray(x) <= 0).astype(np.int32)
    out = np.zeros_like(z, dtype=np.float32); run = 0
    for i, zz in enumerate(z):
        run = run + 1 if zz == 1 else 0
        out[i] = run
    return out

# ----------------------- Label encoders -------------------------------------
train_df = split_store_menu(train_df)
train_df['카테고리'] = train_df['메뉴명'].apply(categorize_menu)

store_le = LabelEncoder(); menu_le = LabelEncoder(); cat_le = LabelEncoder()
train_df['영업장명_le'] = store_le.fit_transform(train_df['영업장명'])
train_df['메뉴명_le']   = menu_le.fit_transform(train_df['메뉴명'])
train_df['카테고리_le'] = cat_le.fit_transform(train_df['카테고리'])
n_stores = len(store_le.classes_); n_menus = len(menu_le.classes_); n_cats = len(cat_le.classes_)
n_dow, n_season = 7, 4

# ----------------------- EXTRA_COLS & scaling -------------------------------
EXTRA_COLS = [
    '평균판매금액','cooking_ratio','1일내장객',
    '일','주차','공휴일여부','월_sin','월_cos','요일_sin','요일_cos',
    'is_weekend','전일_공휴일','익일_공휴일','휴일전날','휴일다음날','연휴중여부',
    'wd_mean_local_28','lag_7','lag_14',
    'roll_mean_1','roll_mean_3','roll_mean_7','roll_mean_14','roll_mean_28',
    'roll_std_3','roll_std_7','roll_std_14','roll_std_28',
    'grp_ski','grp_hwadam','grp_lodging',
]
# Ensure numeric + fill NA
train_df[EXTRA_COLS] = train_df[EXTRA_COLS].apply(pd.to_numeric, errors='coerce')
train_df[EXTRA_COLS] = train_df[EXTRA_COLS].fillna(train_df[EXTRA_COLS].median())
extra_scaler = MinMaxScaler()
train_df[EXTRA_COLS] = extra_scaler.fit_transform(train_df[EXTRA_COLS])

# ----------------------- Final-round metric ---------------------------------
def _mask_nonzero_y(a, p):
    a = np.asarray(a, dtype=float); p = np.asarray(p, dtype=float)
    m = (a != 0) & np.isfinite(a) & np.isfinite(p)
    return a[m], p[m]

def smape_comp(a, p, eps=1e-9):
    a, p = _mask_nonzero_y(a, p)
    if a.size == 0: return 0.0
    return float(np.mean(2*np.abs(a-p)/(np.abs(a)+np.abs(p)+eps)))

def nmae_comp(a, p, eps=1e-9):
    a, p = _mask_nonzero_y(a, p)
    if a.size == 0: return 0.0
    return float(np.mean(np.abs(a-p)) / (np.mean(a)+eps))

def nrmse_comp(a, p, eps=1e-9):
    a, p = _mask_nonzero_y(a, p)
    if a.size == 0: return 0.0
    return float(np.sqrt(np.mean((a-p)**2)) / (np.mean(a)+eps))

def r2_comp(a, p):
    a, p = _mask_nonzero_y(a, p)
    if a.size < 2: return 0.0
    ca=a-a.mean(); cp=p-p.mean()
    denom = np.linalg.norm(ca)*np.linalg.norm(cp)
    if denom==0: return 0.0
    r = float(np.dot(ca, cp)/denom)
    return r*r

def final_score(a, p):
    A = 1.0 - smape_comp(a, p)/2.0           # sMAPE in [0,2] -> normalized
    B = 1.0 - min(1.0, nmae_comp(a, p))
    C = 1.0 - min(1.0, nrmse_comp(a, p))
    D = r2_comp(a, p)
    return 0.25*(A+B+C+D)

# ----------------------- Windows -------------------------------------------
def build_train_val_windows(df):
    train_items, val_items = [], []
    for (st, mn), g in df.groupby(['영업장명','메뉴명'], sort=True):
        g = g.sort_values('영업일자')
        if len(g) < LOOKBACK + HORIZON: continue

        vals = g['매출수량'].astype(np.float32).values
        vals = np.maximum(vals, 0.0)

        d = pd.to_datetime(g['영업일자'])
        dow = d.dt.dayofweek.astype(np.int64).values
        season = d.dt.month.map(lambda m: 0 if m in [12,1,2] else 1 if m in [3,4,5] else 2 if m in [6,7,8] else 3).astype(np.int64).values

        extras = g[EXTRA_COLS].to_numpy(np.float32)

        sid  = int(store_le.transform([st])[0])
        mid  = int(menu_le.transform([mn])[0])
        cid  = int(cat_le.transform([g.iloc[0]['카테고리']])[0])

        total = len(g) - LOOKBACK - HORIZON + 1
        for start in range(total):
            s, e_in, e_all = start, start+LOOKBACK, start+LOOKBACK+HORIZON
            x_raw  = np.maximum(vals[s:e_in], 0.0)
            y_raw  = np.maximum(vals[e_in:e_all], 0.0)
            x_dow  = dow[s:e_in]
            x_sea  = season[s:e_in]
            x_zero = make_zero_streak(x_raw)
            x_extra = extras[s:e_in, :]

            item = {
                'x_vals': np.log1p(x_raw),
                'x_dow': x_dow, 'x_sea': x_sea,
                'x_zero': x_zero, 'x_extra': x_extra,
                'store': sid, 'menu': mid, 'cat': cid,
                'y_val': np.log1p(y_raw),
                'y_mask': (y_raw > 0).astype(np.float32),
                'col': f"{st}_{mn}"
            }
            (val_items if start == total-1 else train_items).append(item)

    if len(val_items) == 0 and len(train_items) > 0:
        cut = min(1024, len(train_items)//5 if len(train_items) >= 5 else 1)
        val_items = train_items[-cut:]; train_items = train_items[:-cut]
    return train_items, val_items

train_items, val_items = build_train_val_windows(train_df)

# ----------------------- Dataset/DataLoader ---------------------------------
class SeqDataset(Dataset):
    def __init__(self, items, has_target=True):
        self.items = items; self.has_target = has_target
    def __len__(self): return len(self.items)
    def __getitem__(self, i):
        it = self.items[i]
        x_vals=torch.tensor(it['x_vals'],dtype=torch.float32)
        x_dow =torch.tensor(it['x_dow'], dtype=torch.long)
        x_sea =torch.tensor(it['x_sea'], dtype=torch.long)
        x_zero=torch.tensor(it['x_zero'],dtype=torch.float32)
        x_extra=torch.tensor(it['x_extra'],dtype=torch.float32)
        # sanitize any weird nums just in case
        x_vals = torch.nan_to_num(x_vals, nan=0.0, posinf=0.0, neginf=0.0)
        x_zero = torch.nan_to_num(x_zero, nan=0.0, posinf=0.0, neginf=0.0)
        x_extra= torch.nan_to_num(x_extra,nan=0.0, posinf=0.0, neginf=0.0)
        store =torch.tensor(it['store'], dtype=torch.long)
        menu  =torch.tensor(it['menu'], dtype=torch.long)
        cat   =torch.tensor(it['cat'], dtype=torch.long)
        if self.has_target:
            y_val =torch.tensor(it['y_val'], dtype=torch.float32)
            y_mask=torch.tensor(it['y_mask'],dtype=torch.float32)
            y_val = torch.nan_to_num(y_val, nan=0.0, posinf=0.0, neginf=0.0)
            return x_vals,x_dow,x_sea,x_zero,x_extra,store,menu,cat,y_val,y_mask,it['col']
        else:
            return x_vals,x_dow,x_sea,x_zero,x_extra,store,menu,cat,it['col']

g = torch.Generator(); g.manual_seed(SEED)
train_dl = DataLoader(SeqDataset(train_items, True), batch_size=BATCH, shuffle=True,
                      num_workers=0, generator=g, persistent_workers=False, drop_last=False)
val_dl   = DataLoader(SeqDataset(val_items, True),   batch_size=BATCH, shuffle=False,
                      num_workers=0, generator=g, persistent_workers=False, drop_last=False)

# ----------------------- Models --------------------------------------------
class LSTMTwoHeadWithCat(nn.Module):
    def __init__(self, n_stores, n_menus, n_cats, n_dow=7, n_season=4,
                 hidden=128, emb_store=16, emb_menu=32, emb_cat=8, emb_dow=4, emb_sea=2,
                 extra_dim=0, horizon=7):
        super().__init__()
        self.store_emb = nn.Embedding(n_stores, emb_store)
        self.menu_emb  = nn.Embedding(n_menus,  emb_menu)
        self.cat_emb   = nn.Embedding(n_cats,   emb_cat)
        self.dow_emb   = nn.Embedding(n_dow,    emb_dow)
        self.sea_emb   = nn.Embedding(n_season, emb_sea)
        in_dim = (1 + 1) + extra_dim + emb_dow + emb_sea + emb_store + emb_menu + emb_cat
        self.lstm = nn.LSTM(in_dim, hidden, batch_first=True)
        self.cls_head = nn.Linear(hidden, horizon)
        self.reg_head = nn.Sequential(nn.Linear(hidden, hidden), nn.ReLU(), nn.Linear(hidden, horizon))
    def forward(self, x_vals, x_dow, x_sea, x_zero, x_extra, store, menu, cat):
        B, L = x_vals.shape
        x = torch.cat([
            x_vals.unsqueeze(-1),
            x_zero.unsqueeze(-1),
            x_extra,
            self.dow_emb(x_dow),
            self.sea_emb(x_sea),
            self.store_emb(store).unsqueeze(1).repeat(1,L,1),
            self.menu_emb(menu).unsqueeze(1).repeat(1,L,1),
            self.cat_emb(cat).unsqueeze(1).repeat(1,L,1),
        ], dim=-1)
        h, _ = self.lstm(x)
        last = h[:, -1, :]
        return self.cls_head(last), self.reg_head(last)

class NHITSEncoder(nn.Module):
    def __init__(self, L=28, latent=128, pools=(1,2,4), blocks=2, hidden=256, drop=0.0):
        super().__init__()
        assert all(L % r == 0 for r in pools)
        self.L = L; self.pools = pools
        self.blocks = nn.ModuleList()
        for r in pools:
            Lp = L // r
            stack = nn.ModuleList()
            for _ in range(blocks):
                stack.append(nn.Sequential(
                    nn.Linear(Lp, hidden), nn.ReLU(),
                    nn.Dropout(drop),
                    nn.Linear(hidden, Lp)
                ))
            self.blocks.append(stack)
        self.head = nn.Linear(L, latent)
    @staticmethod
    def _fixed_avg_pool1d(x, kernel):
        y = F.avg_pool1d(x.unsqueeze(1), kernel_size=kernel, stride=kernel, ceil_mode=False)
        return y.squeeze(1)
    @staticmethod
    def _upsample_nearest(y, out_len):
        z = F.interpolate(y.unsqueeze(1), size=out_len, mode='nearest')
        return z.squeeze(1)
    def forward(self, x):  # x: (B, L)
        r = x
        for r_scale, stack in zip(self.pools, self.blocks):
            z = self._fixed_avg_pool1d(r, kernel=r_scale)
            for blk in stack:
                b = blk(z); z = z - b
            r = r - self._upsample_nearest(b, self.L)
        return self.head(r)

class NHITSTwoHeadWithCat(nn.Module):
    def __init__(self, n_stores, n_menus, n_cats, n_dow=7, n_season=4,
                 emb_store=16, emb_menu=32, emb_cat=8, emb_dow=4, emb_sea=2,
                 extra_dim=0, latent=128, horizon=7):
        super().__init__()
        self.store_emb = nn.Embedding(n_stores, emb_store)
        self.menu_emb  = nn.Embedding(n_menus,  emb_menu)
        self.cat_emb   = nn.Embedding(n_cats,   emb_cat)
        self.dow_emb   = nn.Embedding(n_dow,    emb_dow)
        self.sea_emb   = nn.Embedding(n_season, emb_sea)
        self.ctx_proj   = nn.Linear(emb_dow + emb_sea + emb_store + emb_menu + emb_cat, 1)
        self.extra_proj = nn.Linear(1, 1)                 # zero-streak scalar
        self.extra_num_proj = nn.Linear(extra_dim, 1)     # numeric extras → scalar
        self.enc       = NHITSEncoder(L=LOOKBACK, latent=latent, pools=(1,2,4), blocks=2, hidden=256, drop=0.0)
        self.cls_head  = nn.Linear(latent, horizon)
        self.reg_head  = nn.Sequential(nn.Linear(latent, latent), nn.ReLU(), nn.Linear(latent, horizon))
    def forward(self, x_vals, x_dow, x_sea, x_zero, x_extra, store, menu, cat):
        B, L = x_vals.shape
        ctx = torch.cat([
            self.dow_emb(x_dow), self.sea_emb(x_sea),
            self.store_emb(store).unsqueeze(1).repeat(1,L,1),
            self.menu_emb(menu).unsqueeze(1).repeat(1,L,1),
            self.cat_emb(cat).unsqueeze(1).repeat(1,L,1),
        ], dim=-1)
        ctx_scalar   = self.ctx_proj(ctx).squeeze(-1)                 # (B,L)
        extra_scalar = self.extra_proj(x_zero.unsqueeze(-1)).squeeze(-1)
        extra_num_sc = self.extra_num_proj(x_extra).squeeze(-1)       # (B,L)
        x_in = x_vals + 0.10*ctx_scalar + 0.05*extra_scalar + 0.05*extra_num_sc
        h = self.enc(x_in)
        return self.cls_head(h), self.reg_head(h)

# ----------------------- Train/Eval (composite metric) ----------------------
def evaluate_composite(model, dl, temperature=1.0):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for x_vals,x_dow,x_sea,x_zero,x_extra,store,menu,cat,y_val,y_mask,cols in dl:
            x_vals,x_dow,x_sea = x_vals.to(DEVICE), x_dow.to(DEVICE), x_sea.to(DEVICE)
            x_zero = x_zero.to(DEVICE); x_extra = x_extra.to(DEVICE)
            store,menu,cat = store.to(DEVICE), menu.to(DEVICE), cat.to(DEVICE)
            y_val = y_val.to(DEVICE)
            logit, reg = model(x_vals,x_dow,x_sea,x_zero,x_extra,store,menu,cat)
            prob = torch.sigmoid(logit/temperature)
            pred = (torch.expm1(reg) * prob).clamp_min(1)
            y_true.append(torch.expm1(y_val).cpu().numpy())
            y_pred.append(pred.cpu().numpy())
    yt = np.concatenate(y_true,0).ravel()
    yp = np.concatenate(y_pred,0).ravel()
    return final_score(yt, yp)

def train_model(model, train_dl, val_dl, ckpt_path):
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
    bce = nn.BCEWithLogitsLoss(); huber = nn.SmoothL1Loss()
    best = -1e9; wait = 0
    for ep in range(1, EPOCHS+1):
        model.train(); run=0; n=0
        for x_vals,x_dow,x_sea,x_zero,x_extra,store,menu,cat,y_val,y_mask,_ in train_dl:
            x_vals,x_dow,x_sea = x_vals.to(DEVICE), x_dow.to(DEVICE), x_sea.to(DEVICE)
            x_zero = x_zero.to(DEVICE); x_extra = x_extra.to(DEVICE)
            store,menu,cat = store.to(DEVICE), menu.to(DEVICE), cat.to(DEVICE)
            y_val,y_mask = y_val.to(DEVICE), y_mask.to(DEVICE)

            logit, reg = model(x_vals,x_dow,x_sea,x_zero,x_extra,store,menu,cat)
            loss_cls = bce(logit, y_mask)
            mask = (y_mask>0).float()
            loss_reg = huber(reg*mask, y_val*mask) if mask.sum()>0 else torch.tensor(0.0, device=DEVICE)
            loss = 1.0*loss_cls + 2.0*loss_reg

            optimizer.zero_grad(); loss.backward(); optimizer.step()
            run += loss.item()*x_vals.size(0); n += x_vals.size(0)

        valid_score = evaluate_composite(model, val_dl, temperature=1.0)
        print(f"[{ep:02d}/{EPOCHS}] train_loss={run/max(n,1):.4f}  valid_SCORE={valid_score:.6f}")

        if valid_score > best + 1e-12:
            best = valid_score; wait = 0
            torch.save(model.state_dict(), ckpt_path)
        else:
            wait += 1
            if wait >= PATIENCE:
                print("Early stopping."); break
    print(f"=> Best valid SCORE: {best:.6f}  ({ckpt_path})")
    sd = _load_state_dict_safe(ckpt_path, map_location=DEVICE)
    model.load_state_dict(sd)
    model.eval()
    return best, model

# ----------------------- Train both models ----------------------------------
print("====================================")
print("       Training LSTM Model")
print("====================================")
lstm = LSTMTwoHeadWithCat(n_stores, n_menus, n_cats, n_dow, n_season,
                          extra_dim=len(EXTRA_COLS), horizon=HORIZON).to(DEVICE)
_, lstm = train_model(lstm, train_dl, val_dl, 'best_lstm.pt')

print("====================================")
print("       Training N-HiTS Model")
print("====================================")
nhits = NHITSTwoHeadWithCat(n_stores, n_menus, n_cats, n_dow, n_season,
                            extra_dim=len(EXTRA_COLS), latent=128, horizon=HORIZON).to(DEVICE)
_, nhits = train_model(nhits, train_dl, val_dl, 'best_nhits.pt')

# ----------------------- Temperature tuning ---------------------------------
def tune_temperature(model, dl, grid):
    scores = []
    for t in grid:
        s = evaluate_composite(model, dl, temperature=t)
        scores.append((s, t))
    best_s, best_t = max(scores, key=lambda x: x[0])
    return best_t, best_s, scores

BEST_TEMP_LSTM, LSTM_VAL_AT_BEST_T, _ = tune_temperature(lstm, val_dl, TEMP_GRID)
BEST_TEMP_NHIT, NHIT_VAL_AT_BEST_T, _ = tune_temperature(nhits, val_dl, TEMP_GRID)
print(f"[Temp Tuning] LSTM best τ={BEST_TEMP_LSTM} → valid_SCORE={LSTM_VAL_AT_BEST_T:.6f}")
print(f"[Temp Tuning] N-HiTS best τ={BEST_TEMP_NHIT} → valid_SCORE={NHIT_VAL_AT_BEST_T:.6f}")

# ----------------------- Validation blending --------------------------------
@torch.no_grad()
def collect_val_preds(model, dl, temperature=1.0):
    model.eval()
    col_list, y_list, p_list = [], [], []
    for x_vals,x_dow,x_sea,x_zero,x_extra,store,menu,cat,y_val,y_mask,cols in dl:
        x_vals,x_dow,x_sea = x_vals.to(DEVICE), x_dow.to(DEVICE), x_sea.to(DEVICE)
        x_zero = x_zero.to(DEVICE); x_extra = x_extra.to(DEVICE)
        store,menu,cat = store.to(DEVICE), menu.to(DEVICE), cat.to(DEVICE)
        logit, reg = model(x_vals,x_dow,x_sea,x_zero,x_extra,store,menu,cat)
        prob = torch.sigmoid(logit/temperature)
        pred = (torch.expm1(reg) * prob).clamp_min(1)
        col_list += list(cols)
        y_list.append(torch.expm1(y_val).cpu().numpy())
        p_list.append(pred.cpu().numpy())
    y = np.concatenate(y_list,0); p = np.concatenate(p_list,0)
    return np.array(col_list), y, p

cols_v, y_v, pL_v = collect_val_preds(lstm, val_dl, temperature=BEST_TEMP_LSTM)
_,     _, pN_v    = collect_val_preds(nhits, val_dl, temperature=BEST_TEMP_NHIT)

def col_to_groupkey(colname):
    store, menu = colname.split('_', 1)
    cat = categorize_menu(menu)
    if GROUPING == 'store': return store
    elif GROUPING == 'category': return cat
    elif GROUPING == 'store_category': return f"{store}_{cat}"
    else: return (store, cat)

keys_v = np.array([col_to_groupkey(c) for c in cols_v])

def composite_arrays(y_true, yL, yN, a):
    yp = a*yL + (1-a)*yN
    return final_score(y_true.ravel(), yp.ravel())

global_blend_score = composite_arrays(y_v, pL_v, pN_v, GLOBAL_ALPHA)
print(f"[Validation SCORE] LSTM(τ*)={LSTM_VAL_AT_BEST_T:.6f}  N-HiTS(τ*)={NHIT_VAL_AT_BEST_T:.6f}  Global Blend a=0.80: {global_blend_score:.6f}")

group_alpha = {}
for g in np.unique(keys_v):
    idx = np.where(keys_v == g)[0]
    if idx.size < MIN_ITEMS_PER_GROUP: continue
    y = y_v[idx]; l = pL_v[idx]; n = pN_v[idx]
    best_s = -1e9; best_a = GLOBAL_ALPHA
    for a in ALPHAS_GRID:
        s = composite_arrays(y, l, n, a)
        if s > best_s:
            best_s, best_a = s, a
    group_alpha[g] = best_a
print(f"Tuned groups (alpha): {len(group_alpha)} / {len(np.unique(keys_v))}")

def apply_group_blend(cols, pL, pN, default_a=GLOBAL_ALPHA):
    out = np.zeros_like(pL)
    for i, c in enumerate(cols):
        gkey = col_to_groupkey(c)
        a = group_alpha.get(gkey, default_a)
        out[i] = a * pL[i] + (1-a) * pN[i]
    return out

p_group = apply_group_blend(cols_v, pL_v, pN_v, GLOBAL_ALPHA)
group_blend_score = final_score(y_v.ravel(), p_group.ravel())
print("======== Validation Summary ========")
print(f"LSTM (best τ={BEST_TEMP_LSTM}): {LSTM_VAL_AT_BEST_T:.6f}")
print(f"N-HiTS (best τ={BEST_TEMP_NHIT}): {NHIT_VAL_AT_BEST_T:.6f}")
print(f"Global Blend (α=0.80): {global_blend_score:.6f}")
print(f"Group Blend (tuned α): {group_blend_score:.6f}")

# ----------------------- Test inference ------------------------------------
def preprocess_test_build_features(raw_df: pd.DataFrame, sid_tag: str) -> pd.DataFrame:
    key_col = CANON_KEY; target="매출수량"
    price_raw  = read_csv_safe(PRICE_PATH)
    price_slim = preprocess_price(price_raw)
    df = normalize_key_column(raw_df)
    df[target] = pd.to_numeric(df.get(target, 0), errors='coerce').fillna(0).clip(lower=0)
    df = df.merge(price_slim, on=key_col, how="left")

    trot = build_room_ratio(os.path.join(TEST_meta_DIR, f"TEST_room_{sid_tag}.csv"), room_type_meta)
    df  = attach_ratio(df, trot)

    tsdf = read_csv_safe(os.path.join(TEST_meta_DIR, f"TEST_ski_{sid_tag}.csv"))
    df  = attach_daily_ski(df, tsdf)

    df = df.sort_values([key_col, "영업일자"])
    df = finalize_features(df)
    dfX = add_venue_group_features(df.drop(columns=[target], errors="ignore"), col=key_col, keep_venue_name=True)

    # numeric extras
    dfX[EXTRA_COLS] = dfX[EXTRA_COLS].apply(pd.to_numeric, errors='coerce').fillna(0)
    dfX[EXTRA_COLS] = extra_scaler.transform(dfX[EXTRA_COLS])
    dfX[target] = pd.to_numeric(df[target], errors='coerce').fillna(0).clip(lower=0)
    return dfX

@torch.no_grad()
def predict7(model, g_seq, sid, mid, cid, temperature=1.0):
    g_seq = g_seq.sort_values('영업일자')
    raw  = g_seq['매출수량'].values.astype(np.float32) if '매출수량' in g_seq.columns else np.zeros(LOOKBACK, np.float32)
    raw  = np.maximum(raw, 0.0)
    vals = np.log1p(raw)
    d    = pd.to_datetime(g_seq['영업일자'])
    dows = d.dt.dayofweek.astype(np.int64).values
    seas = d.dt.month.map(lambda m: 0 if m in [12,1,2] else 1 if m in [3,4,5] else 2 if m in [6,7,8] else 3).astype(np.int64).values
    x_extra = g_seq[EXTRA_COLS].to_numpy(np.float32)
    if len(vals) != LOOKBACK:
        return np.ones(HORIZON, dtype=np.float32)
    x_zero = make_zero_streak(raw)

    x_vals = torch.tensor(vals[None,:], dtype=torch.float32).to(DEVICE)
    x_dow  = torch.tensor(dows[None,:], dtype=torch.long).to(DEVICE)
    x_sea  = torch.tensor(seas[None,:], dtype=torch.long).to(DEVICE)
    x_zero = torch.tensor(x_zero[None,:], dtype=torch.float32).to(DEVICE)
    x_extra= torch.tensor(x_extra[None,:,:], dtype=torch.float32).to(DEVICE)
    store  = torch.tensor([sid], dtype=torch.long).to(DEVICE)
    menu   = torch.tensor([mid], dtype=torch.long).to(DEVICE)
    cat    = torch.tensor([cid], dtype=torch.long).to(DEVICE)
    logit, reg = model(x_vals,x_dow,x_sea,x_zero,x_extra,store,menu,cat)
    prob = torch.sigmoid(logit/temperature)
    pred = (torch.expm1(reg) * prob).clamp_min(1)
    return pred.cpu().numpy().ravel()

def group_key_from_names(store, menu):
    cat = categorize_menu(menu)
    if GROUPING == 'store': return store
    elif GROUPING == 'category': return cat
    else: return f"{store}_{cat}"

# ---- Build submission (long & wide) ----
sub_rows = []
if os.path.exists(SAMPLE_PATH):
    for path in sorted(glob.glob(os.path.join(TEST_DIR, "TEST_*.csv"))):
        t_raw = read_csv_safe(path, parse_dates=["영업일자"])
        t_raw = normalize_key_column(t_raw)
        tag   = re.search(r'TEST_(\d+)', os.path.basename(path)).group(1)
        t_df  = preprocess_test_build_features(t_raw, sid_tag=tag)

        # encode ids
        t_df = split_store_menu(t_df)
        t_df['카테고리'] = t_df['메뉴명'].apply(categorize_menu)
        def map_safe(le, values):
            mp = {c:i for i,c in enumerate(le.classes_)}
            return np.array([mp.get(v, 0) for v in values], dtype=np.int64)
        t_df['영업장명_le'] = map_safe(store_le, t_df['영업장명'].values)
        t_df['메뉴명_le']   = map_safe(menu_le,  t_df['메뉴명'].values)
        t_df['카테고리_le'] = map_safe(cat_le,   t_df['카테고리'].values)

        for (sid, mid, cid), g in t_df.groupby(['영업장명_le','메뉴명_le','카테고리_le'], sort=True):
            g28 = g.sort_values('영업일자').tail(LOOKBACK)
            if len(g28) < LOOKBACK: continue
            yL = predict7(lstm, g28, sid, mid, cid, temperature=BEST_TEMP_LSTM)
            yN = predict7(nhits, g28, sid, mid, cid, temperature=BEST_TEMP_NHIT)

            store_name = g28.iloc[-1]['영업장명']
            menu_name  = g28.iloc[-1]['메뉴명']
            gkey = group_key_from_names(store_name, menu_name)
            a = group_alpha.get(gkey, GLOBAL_ALPHA)
            y = np.maximum(a*yL + (1-a)*yN, 1.0)

            sm_name = f"{store_name}_{menu_name}"
            for i, v in enumerate(y, 1):
                sub_rows.append({'영업일자': f'TEST_{tag}+{i}일', CANON_KEY: sm_name, '매출수량': float(v)})
else:
    print("sample_submission.csv not found; skipped test inference.")

pred_long = pd.DataFrame(sub_rows)
long_path = os.path.join(DATA_PATH, "submission_groupblend_long.csv")
if len(pred_long):
    pred_long.to_csv(long_path, index=False)
    print(f"[Saved] {long_path}  shape={pred_long.shape}")
else:
    print("No predictions were generated. Check TEST_*.csv availability.")

def save_wide_aligned(sample_path, sub_long, out_path):
    if not os.path.exists(sample_path): return False, None
    sample = pd.read_csv(sample_path)
    sample = sanitize_columns(sample)
    if '영업일자' in sample.columns and sample.shape[1] > 3:
        wide = sub_long.pivot_table(index='영업일자',
                                    columns=CANON_KEY,
                                    values='매출수량',
                                    aggfunc='first')
        items = list(sample.columns[1:]); dates = sample['영업일자']
        wide = wide.reindex(index=dates, columns=items)
        wide = wide.apply(pd.to_numeric, errors='coerce').fillna(1.0)
        wide = np.clip(wide, 1.0, None)
        out = pd.concat([sample[['영업일자']].reset_index(drop=True),
                         pd.DataFrame(wide, columns=items).reset_index(drop=True)], axis=1)
        out.to_csv(out_path, index=False)
        return True, out.shape
    return False, None

saved_wide, wide_shape = save_wide_aligned(SAMPLE_PATH, pred_long, SUBMIT_PATH)
if saved_wide:
    print(f"[Saved] {SUBMIT_PATH} (WIDE aligned) shape={wide_shape}")
else:
    print("Sample is not WIDE or not found → only long CSV saved.")

Device: cuda
       Training LSTM Model
[01/25] train_loss=0.7457  valid_SCORE=0.374071
[02/25] train_loss=0.5709  valid_SCORE=0.403262
[03/25] train_loss=0.5331  valid_SCORE=0.415332
[04/25] train_loss=0.5053  valid_SCORE=0.422289
[05/25] train_loss=0.4837  valid_SCORE=0.428581
[06/25] train_loss=0.4671  valid_SCORE=0.453441
[07/25] train_loss=0.4526  valid_SCORE=0.435822
[08/25] train_loss=0.4388  valid_SCORE=0.442384
[09/25] train_loss=0.4262  valid_SCORE=0.450633
[10/25] train_loss=0.4132  valid_SCORE=0.462772
[11/25] train_loss=0.4018  valid_SCORE=0.472433
[12/25] train_loss=0.3910  valid_SCORE=0.446792
[13/25] train_loss=0.3812  valid_SCORE=0.458323
[14/25] train_loss=0.3703  valid_SCORE=0.457380
[15/25] train_loss=0.3596  valid_SCORE=0.449387
[16/25] train_loss=0.3507  valid_SCORE=0.475510
[17/25] train_loss=0.3420  valid_SCORE=0.458952
[18/25] train_loss=0.3332  valid_SCORE=0.465481
[19/25] train_loss=0.3253  valid_SCORE=0.482427
[20/25] train_loss=0.3175  valid_SCORE=0.477867


# 머신

In [7]:
# ============================================================
# 단일 셀: 메뉴별 매출수량 예측 (피처링: 날씨 + 시계열 + 시간/캘린더 + 메뉴카테고리 + RFM)
# ============================================================

# -------------------------
# 0) 기본 설정/임포트
# -------------------------
import os, re, glob, warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
pd.options.display.float_format = "{:.4f}".format
np.random.seed(42)

try:
    from IPython.display import display
except Exception:
    def display(x): print(x)

import matplotlib.pyplot as plt
import holidays  # 공휴일
from sklearn.preprocessing import RobustScaler
from pathlib import Path

# ------------------------------------------------
# 1) 경로
# ------------------------------------------------
BASE_DIR = Path("../data")
TRAIN_DIR = BASE_DIR / "train"
TEST_DIR  = BASE_DIR / "test"

TRAIN_PATH        = TRAIN_DIR / "train.csv"
SAMPLE_SUB_PATH   = BASE_DIR / "sample_submission.csv"
TRAIN_META_DIR    = TRAIN_DIR / "meta"
TEST_META_DIR     = TEST_DIR / "meta"
TEST_GLOB         = sorted([p.name for p in TEST_DIR.glob("TEST_*.csv")])

WEATHER_DIR = Path("weather")

# ------------------------------------------------
# 2) 유틸(평가지표·시간/캘린더·카테고리·시계열·클렌징)
# ------------------------------------------------
def group_smape_by_store_with_detail(df, actual_col='매출수량', pred_col='예측값', group_col='업장명'):
    smape_per_store = {}
    eps = 1e-9
    for store, g in df.groupby(group_col):
        gg = g[g[actual_col] != 0]
        if len(gg)==0:
            s = np.nan
        else:
            num = np.abs(gg[actual_col]-gg[pred_col])
            den = np.abs(gg[actual_col]) + np.abs(gg[pred_col]) + eps
            s = (2 * num / den).mean()
        smape_per_store[store] = float(s) if np.isfinite(s) else np.nan
    overall_mean = float(np.nanmean(list(smape_per_store.values()))) if smape_per_store else np.nan
    return smape_per_store, overall_mean

def smape_group_macro(df, actual_col='매출수량', pred_col='예측값', group_col='업장명'):
    _, overall = group_smape_by_store_with_detail(df, actual_col, pred_col, group_col)
    return overall

# ---- 시간/캘린더/메뉴 카테고리 ----
def add_time_features(df, date_col='영업일자'):
    out = df.copy()
    dt = out[date_col].dt
    month = dt.month
    out['일']   = dt.day
    out['요일'] = dt.dayofweek
    out['주차'] = dt.isocalendar().week.astype('int32')
    out['계절'] = month.map(lambda m: 0 if m in [12, 1, 2]
                                      else 1 if m in [3, 4, 5]
                                      else 2 if m in [6, 7, 8]
                                      else 3)
    kr_years = dt.year.unique()
    kr_holidays = holidays.KR(years=kr_years)
    # 타입 보정: date 비교 사용
    out['공휴일여부'] = out[date_col].dt.date.isin(kr_holidays).astype(int)
    out['월_sin']   = np.sin(2*np.pi*month/12)
    out['월_cos']   = np.cos(2*np.pi*month/12)
    out['요일_sin'] = np.sin(2*np.pi*out['요일']/7)
    out['요일_cos'] = np.cos(2*np.pi*out['요일']/7)
    return out

def add_calendar_context_features(df: pd.DataFrame, date_col: str = '영업일자') -> pd.DataFrame:
    out = df.copy()
    d = out[date_col].dt
    if 'is_weekend' not in out.columns:
        out['is_weekend'] = (d.dayofweek >= 5).astype(int)

    years = sorted(out[date_col].dt.year.unique())
    kr_hdays = set(holidays.KR(years=years).keys())
    prev_day = (out[date_col] - pd.Timedelta(days=1)).dt.date
    next_day = (out[date_col] + pd.Timedelta(days=1)).dt.date
    out['전일_공휴일'] = prev_day.map(lambda x: int(x in kr_hdays))
    out['익일_공휴일'] = next_day.map(lambda x: int(x in kr_hdays))

    if '공휴일여부' in out.columns:
        out['휴일전날']   = ((out['공휴일여부'] == 0) & (out['익일_공휴일'] == 1)).astype(int)
        out['휴일다음날'] = ((out['공휴일여부'] == 0) & (out['전일_공휴일'] == 1)).astype(int)
        out['연휴중여부'] = (((out['전일_공휴일'] == 1) | (out['익일_공휴일'] == 1)) & (out['공휴일여부'] == 1)).astype(int)
    else:
        out['휴일전날']   = (out['익일_공휴일'] == 1).astype(int)
        out['휴일다음날'] = (out['전일_공휴일'] == 1).astype(int)
        out['연휴중여부'] = ((out['전일_공휴일'] == 1) | (out['익일_공휴일'] == 1)).astype(int)
    return out

def classify_menu_features(df, col='메뉴명'):
    df = df.copy()
    name_col = df[col].astype(str).str.lower().str.replace(" ", "", regex=False)
    categories = {
        '일회용품': ['일회용', '접시', '수저', '종이컵', '컵'],
        'BBQ' : ['돈육구이', '킬바사소세지', 'bbq', '양갈비', '삼겹살추가', '200g', '본삼겹'],
        '한식': ['김치', '된장', '비빔', '국밥', '찌개', '순대', '파전', '돌솥', '설렁탕', '해장국', '한우', '냉면', '갱시기', '양지탕', '갈비', '골뱅이', '닭발','정식'],
        '중식': ['짜장', '짬뽕', '마라','볶음밥'],
        '양식': ['파스타', '돈까스', '샐러드', '스테이크', '리조또', '피자', '햄버거', '치즈프라이', '알리오', '까르보나라', '스파게티', '플래터', '브런치', '패키지'],
        '간식' : ['떡볶이', '신라면', '사발면', '페스츄리소시지', '꼬치어묵', '핫도그', '우동'],
        '음료': ['아메리카노', '라떼', '에이드', '주스', '식혜', '차', '커피', '티', '콜라', 'coffee', '스프라이트', '미숫가루', '생수'],
        '주류': ['소주', '카스', '참이슬', '막걸리', '하이네켄', '버드와이저', '맥주', '테라', '칵테일', 'wine', '하이볼', '토닉', 'gls', 'beer', '처음처럼', '와인', '미션서드카베르네쉬라', '스텔라', '샷'],
        '디저트': ['아이스크림', '쿠키', '디저트', '케이크', '스크림'],
        '식자재': ['공깃밥', '햇반', '쌈장', '소스', '사리', '라면사리', '쌈야채', '주먹밥', '허브솔트', '빵추가', '야채추가'],
        '대여': ['대여', '이용료', '의자', '룸', 'conference', 'hall', 'ballroom', 'opus']
    }
    def map_category(name):
        for cat, keywords in categories.items():
            for kw in keywords:
                if kw in name: return cat
        return '기타'
    df['menu_category'] = name_col.apply(map_category)
    return df

# ---- 시계열 ----
def add_filtered_ts_features(
    df: pd.DataFrame,
    group_cols=('영업장명_메뉴명',),
    date_col: str = '영업일자',
    target_col: str = '매출수량',
    lags=(7, 14),
    roll_mean_windows=(1, 3, 7, 14, 28),
    roll_std_windows=(3, 7, 14, 28),
    use_nz_ratio_28=True,
    use_trend_7_28=False,
    use_lag7_diff14=False,
    use_streak_pos=True,
):
    if isinstance(group_cols, str):
        group_cols = (group_cols,)
    out = df.sort_values(list(group_cols) + [date_col]).copy()
    g = out.groupby(list(group_cols))[target_col]
    prev = g.shift(1)
    keys = [out[c] for c in group_cols]
    for k in lags:
        out[f'lag_{k}'] = g.shift(k)
    for w in roll_mean_windows:
        out[f'roll_mean_{w}'] = (
            prev.groupby(keys).rolling(w, min_periods=1).mean()
                .reset_index(level=list(range(len(group_cols))), drop=True)
        )
    for w in roll_std_windows:
        out[f'roll_std_{w}'] = (
            prev.groupby(keys).rolling(w, min_periods=2).std()
                .reset_index(level=list(range(len(group_cols))), drop=True)
        )
    if use_nz_ratio_28:
        nz = (prev > 0).astype(float)
        out['nz_ratio_28'] = (
            nz.groupby(keys).rolling(28, min_periods=1).mean()
              .reset_index(level=list(range(len(group_cols))), drop=True)
        )
    if use_trend_7_28 and {'roll_mean_7','roll_mean_28'} <= set(out.columns):
        out['trend_7_28'] = out['roll_mean_7'] / (out['roll_mean_28'] + 1e-8) - 1
    if use_lag7_diff14 and {'lag_7','lag_14'} <= set(out.columns):
        out['lag_7_diff_14'] = out['lag_7'] - out['lag_14']
    if use_streak_pos:
        prev_pos = (prev.fillna(0) > 0)
        def _streak(arr_bool: pd.Series) -> pd.Series:
            grp_key = (~arr_bool).cumsum()
            return arr_bool.groupby(grp_key).cumsum()
        out['streak_pos'] = prev_pos.groupby(keys).transform(_streak).fillna(0).astype('int32')
    return out

def _add_wd_mean_local_28(g: pd.DataFrame) -> pd.DataFrame:
    g = g.sort_values('영업일자').copy()
    s = pd.to_numeric(g['매출수량'], errors='coerce')
    same_wd_mean = s.shift(7).rolling(4, min_periods=1).mean()
    all_wd_mean  = s.shift(1).rolling(7, min_periods=1).mean()
    wd = same_wd_mean.copy()
    wd = wd.where(~wd.isna(), all_wd_mean).fillna(0.0)
    g['wd_mean_local_28'] = wd.astype(float)
    return g

def sanitize_numeric(df: pd.DataFrame) -> pd.DataFrame:
    num_cols = df.select_dtypes(include=['number', 'bool']).columns
    df[num_cols] = (df[num_cols].replace([np.inf, -np.inf], np.nan).fillna(0.0))
    df[num_cols] = df[num_cols].astype(float)
    return df
# ------------------------------------------------
# 2.6) Global RFM (train 기준으로만 계산 후 전체에 머지)
# ------------------------------------------------
def generate_rfm_features(
    df: pd.DataFrame,
    key_col: str = '영업장명_메뉴명',
    date_col: str = '영업일자',
    qty_col: str = '매출수량',
    기준일자: pd.Timestamp | None = None,
    fallback_recency_days: int | None = None,
    fallback_meanlead_days: int | None = None,
) -> pd.DataFrame:
    """
    Global RFM: df(=train) 전구간에서 key 단위 R/F/M 메타 피처 생성
    - recency/meanlead: 미관측 key는 train 기간 길이로 채움
    - freq/sum/avgqty: 미관측 key는 0
    """
    assert date_col in df.columns and qty_col in df.columns and key_col in df.columns

    # 전역 기본값
    train_min = pd.to_datetime(df[date_col].min())
    train_max = pd.to_datetime(df[date_col].max())
    if 기준일자 is None:
        기준일자 = train_max

    total_span_days = int((train_max - train_min).days)
    if fallback_recency_days is None:
        fallback_recency_days = total_span_days
    if fallback_meanlead_days is None:
        fallback_meanlead_days = total_span_days

    keys_all = df[[key_col]].drop_duplicates()
    pos = df[df[qty_col] > 0].copy()

    if len(pos) > 0:
        last_dt   = pos.groupby(key_col)[date_col].max()
        recency   = (기준일자 - last_dt).dt.days.rename('rfm_recency')

        freq      = pos.groupby(key_col)[date_col].nunique().rename('rfm_freq_days')
        summ      = pos.groupby(key_col)[qty_col].sum().rename('rfm_sum_qty')

        diffs     = (pos.sort_values([key_col, date_col])
                        .assign(_diff=lambda x: x.groupby(key_col)[date_col].diff().dt.days))
        meanlead  = diffs.groupby(key_col)['_diff'].mean().rename('rfm_meanlead')

        rfm = pd.concat([recency, freq, summ, meanlead], axis=1).reset_index()
    else:
        rfm = pd.DataFrame(columns=[key_col, 'rfm_recency','rfm_freq_days','rfm_sum_qty','rfm_meanlead'])

    rfm = keys_all.merge(rfm, on=key_col, how='left')

    rfm['rfm_recency']   = rfm['rfm_recency'].fillna(fallback_recency_days).clip(lower=0)
    rfm['rfm_meanlead']  = rfm['rfm_meanlead'].fillna(fallback_meanlead_days).clip(lower=0)
    rfm['rfm_freq_days'] = rfm['rfm_freq_days'].fillna(0).clip(lower=0)
    rfm['rfm_sum_qty']   = rfm['rfm_sum_qty'].fillna(0.0).clip(lower=0.0)

    rfm['rfm_avgqty'] = (rfm['rfm_sum_qty'] / rfm['rfm_freq_days'].replace(0, np.nan))
    rfm['rfm_avgqty'] = rfm['rfm_avgqty'].replace([np.inf, -np.inf], np.nan).fillna(0.0)

    return rfm[[key_col, 'rfm_recency', 'rfm_freq_days', 'rfm_sum_qty',
                'rfm_meanlead', 'rfm_avgqty']]

# -------------------------
# 3) 데이터 로드
# -------------------------
train = pd.read_csv(TRAIN_PATH, parse_dates=['영업일자'])
train['매출수량'] = pd.to_numeric(train['매출수량'], errors='coerce').clip(lower=0)
train['영업장명_메뉴명'] = train['영업장명_메뉴명'].astype(str).str.strip()
train[['업장명', '메뉴명']] = train['영업장명_메뉴명'].str.split('_', n=1, expand=True)

test_input_dfs = []
for fname in TEST_GLOB:
    fpath = TEST_DIR / fname
    df = pd.read_csv(fpath, parse_dates=['영업일자'])
    df['test_id'] = os.path.splitext(fname)[0]
    df['영업장명_메뉴명'] = df['영업장명_메뉴명'].astype(str).str.strip()
    if '매출수량' in df.columns:
        df['매출수량'] = pd.to_numeric(df['매출수량'], errors='coerce').clip(lower=0)
    df[['업장명', '메뉴명']] = df['영업장명_메뉴명'].str.split('_', n=1, expand=True)
    test_input_dfs.append(df)
test_input_df = pd.concat(test_input_dfs, ignore_index=True)

submission_df = pd.read_csv(SAMPLE_SUB_PATH)
sub_long = submission_df.melt(id_vars='영업일자', var_name='영업장명_메뉴명', value_name='예측값')
sub_long[['업장명', '메뉴명']] = sub_long['영업장명_메뉴명'].astype(str).str.split('_', n=1, expand=True)
sub_long['test_id'] = sub_long['영업일자'].str.extract(r'(TEST_\d+)')
sub_long['offset'] = sub_long['영업일자'].str.extract(r'\+(\d+)일').astype(int)

test_last_dates = {}
for fname in TEST_GLOB:
    fpath = TEST_DIR / fname
    df = pd.read_csv(fpath, parse_dates=['영업일자'])
    test_last_dates[os.path.splitext(fname)[0]] = df['영업일자'].max()

sub_long['기준일자'] = sub_long['test_id'].map(test_last_dates)
sub_long['영업일자'] = sub_long['기준일자'] + pd.to_timedelta(sub_long['offset'], unit='D')
test_pred_df = sub_long[['test_id', '영업일자', '업장명', '메뉴명', '영업장명_메뉴명']].copy()

# -------------------------
# 4) 피처링 (날씨 + 시계열 + 시간/캘린더 + 메뉴카테고리)
# -------------------------
# (4.2) wd_mean_local_28
if '매출수량' not in test_input_df.columns:
    test_input_df['매출수량'] = 0.0
train         = train.groupby('영업장명_메뉴명', group_keys=False).apply(_add_wd_mean_local_28)
test_input_df = test_input_df.groupby(['test_id','영업장명_메뉴명'], group_keys=False).apply(_add_wd_mean_local_28)

# (4.3) 시계열 파생
train_ts = add_filtered_ts_features(
    train, group_cols='영업장명_메뉴명',
    lags=(7,14), roll_mean_windows=(1,3,7,14,28), roll_std_windows=(3,7,14,28),
    use_nz_ratio_28=True, use_trend_7_28=False, use_lag7_diff14=False, use_streak_pos=True
)
input_ts = add_filtered_ts_features(
    test_input_df, group_cols=('test_id','영업장명_메뉴명'),
    lags=(7,14), roll_mean_windows=(1,3,7,14,28), roll_std_windows=(3,7,14,28),
    use_nz_ratio_28=True, use_trend_7_28=False, use_lag7_diff14=False, use_streak_pos=True
)

KEEP_PREFIX = ('lag_', 'roll_mean_', 'roll_std_', 'nz_ratio_', 'streak_')
ts_cols = sorted(set([c for c in train_ts.columns if c.startswith(KEEP_PREFIX)]) |
                 set([c for c in input_ts.columns if c.startswith(KEEP_PREFIX)]))

train = train.merge(
    train_ts[['영업장명_메뉴명','영업일자'] + ts_cols],
    on=['영업장명_메뉴명','영업일자'], how='left'
)
test_input_df = test_input_df.merge(
    input_ts[['test_id','영업장명_메뉴명','영업일자'] + ts_cols],
    on=['test_id','영업장명_메뉴명','영업일자'], how='left'
)

# (4.4) 시간/캘린더 + 메뉴카테고리
train         = add_time_features(train)
test_input_df = add_time_features(test_input_df)
test_pred_df  = add_time_features(test_pred_df)

train         = add_calendar_context_features(train)
test_input_df = add_calendar_context_features(test_input_df)
test_pred_df  = add_calendar_context_features(test_pred_df)

train         = classify_menu_features(train)
test_input_df = classify_menu_features(test_input_df)
test_pred_df  = classify_menu_features(test_pred_df)

# (4.5) 예측 DF에 최신 시계열/주중효과 전달
latest_ts = (
    test_input_df.sort_values('영업일자')
    .groupby(['test_id','영업장명_메뉴명']).tail(1)[['test_id','영업장명_메뉴명'] + ts_cols]
)
test_pred_df = test_pred_df.merge(latest_ts, on=['test_id','영업장명_메뉴명'], how='left')

# wd_mean_local_28 최신값 전달
train = (train.groupby('영업장명_메뉴명', group_keys=False).apply(_add_wd_mean_local_28))
test_input_df = (test_input_df.groupby(['test_id','영업장명_메뉴명'], group_keys=False).apply(_add_wd_mean_local_28))
latest_wd = (test_input_df.sort_values('영업일자')
             .groupby(['test_id','영업장명_메뉴명']).tail(1)[['test_id','영업장명_메뉴명','wd_mean_local_28']])
test_pred_df = test_pred_df.merge(latest_wd, on=['test_id','영업장명_메뉴명'], how='left')

# 전역 수치 클렌징 1차
train        = sanitize_numeric(train)
test_input_df= sanitize_numeric(test_input_df)
test_pred_df = sanitize_numeric(test_pred_df)

# -------------------------
# 4.6) Global RFM 계산(오직 train) 후 전 세트에 머지
# -------------------------
rfm_train = generate_rfm_features(
    train[['영업장명_메뉴명','영업일자','매출수량']].copy(),
    기준일자=train['영업일자'].max(),
).rename(columns={
    'rfm_recency':'rfm_recency_train',
    'rfm_freq_days':'rfm_freq_train',
    'rfm_sum_qty':'rfm_sum_train',
    'rfm_meanlead':'rfm_meanlead_train',
    'rfm_avgqty':'rfm_avgqty_train'
})

train         = train.merge(rfm_train, on='영업장명_메뉴명', how='left')
test_input_df = test_input_df.merge(rfm_train, on='영업장명_메뉴명', how='left')
test_pred_df  = test_pred_df.merge(rfm_train, on='영업장명_메뉴명', how='left')

# 결측 방어 + 전역 수치 클렌징 2차
for df_ in (train, test_input_df, test_pred_df):
    for c in ['rfm_recency_train','rfm_meanlead_train']:
        if c in df_.columns: df_[c] = df_[c].fillna(999)
    for c in ['rfm_freq_train','rfm_sum_train','rfm_avgqty_train']:
        if c in df_.columns: df_[c] = df_[c].fillna(0.0)
train        = sanitize_numeric(train)
test_input_df= sanitize_numeric(test_input_df)
test_pred_df = sanitize_numeric(test_pred_df)

# 무결성 체크(선택): RFM은 test_id와 무관해야 함
_chk = (test_input_df.groupby('영업장명_메뉴명')[['rfm_recency_train','rfm_sum_train']]
        .nunique())
if (_chk > 1).any().any():
    raise RuntimeError("Global RFM anomaly: _train 피처가 test_id에 따라 달라졌습니다.")

# -------------------------
# 5) 입력 피처 선택 + 스케일링
# -------------------------
TARGET = '매출수량'
base_drop_cols = ['영업일자', '영업장명_메뉴명', '업장명', '메뉴명']  # 원천 문자 컬럼 제거

# 과거 메타 잔재는 안전 드롭(없어도 errors='ignore')
DROP_FEATS = [
    "venue_group_ski","venue_group_hwadam","venue_group_lodging","venue_group_other",
    "roll_mean_14",
    "roll_std_14",
    'rfm_sum_train',
    'roll_mean_7',
    '평균판매금액', 
    'ski_visitors', 
    'hwadam_visitors', 
    'cooking_ratio',
    "최고기온(°C)","일강수량(mm)"
]

X_train_tmp = (train.drop(columns=[TARGET] + base_drop_cols + DROP_FEATS, errors='ignore')
                    .select_dtypes(include=['number','bool']))
expected_features = X_train_tmp.columns.tolist()

# 시계열 핵심만 Robust Scaling (날씨/캘린더/RFM은 비스케일)
MASTER_SCALER_ORDER = [
    'lag_7','lag_14',
    'roll_mean_1','roll_mean_3','roll_mean_28',
    'roll_std_3','roll_std_7','roll_std_28',
    'streak_pos', 
]
scaler_cols = [c for c in MASTER_SCALER_ORDER if c in expected_features]

if scaler_cols:
    scaler = RobustScaler().fit(train[scaler_cols])
    train.loc[:, scaler_cols] = scaler.transform(train[scaler_cols])
else:
    scaler = None
dyn_keep = set(scaler_cols)

print(f"[INFO] 최종 피처 개수: {len(expected_features)}")
print(expected_features)


# 전처리/모델 (기본)
from sklearn.preprocessing import OneHotEncoder, RobustScaler, TargetEncoder
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet,
    HuberRegressor, SGDRegressor, PoissonRegressor, TweedieRegressor
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor, ExtraTreesRegressor,
    GradientBoostingRegressor, HistGradientBoostingRegressor,
    AdaBoostRegressor, BaggingRegressor
)
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

# -------------------------
# 6) 모델 ZOO
# -------------------------
def make_model_zoo(random_state=42):
    rs = random_state
    zoo = {}
    zoo["Elastic"]      = ("ElasticNet",    lambda: ElasticNet(alpha=0.001, l1_ratio=0.5, random_state=rs, max_iter=5000))

    try:
        from lightgbm import LGBMRegressor
        base_lgbm = dict(
            n_estimators=1000, learning_rate=0.05,
            num_leaves=63, subsample=0.8, colsample_bytree=0.8,
            n_jobs=-1, random_state=rs
        )
        zoo["LGBM_Tweedie"] = ("LGBM-Tweedie",    lambda: LGBMRegressor(objective="tweedie",
                                                                        tweedie_variance_power=1.2, **base_lgbm))
    except Exception:
        pass
    try:
        from xgboost import XGBRegressor
        base_xgb = dict(
            n_estimators=900, max_depth=8, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            tree_method="hist", n_jobs=-1, random_state=rs
        )
        zoo["XGB_Poisson"]  = ("XGB-Poisson",     lambda: XGBRegressor(objective="count:poisson", **base_xgb))
    except Exception:
        pass
    return zoo

# -------------------------
# 7) AR 동적 피처 & 학습/예측 유틸
# -------------------------
def _model_feature_names(model, fallback_cols):
    if hasattr(model, "_feature_list") and getattr(model, "_feature_list"):
        return list(model._feature_list)
    if hasattr(model, 'feature_name_') and getattr(model, 'feature_name_', None):
        return list(model.feature_name_)
    try:
        return list(model.booster_.feature_name())
    except Exception:
        return list(fallback_cols)

def train_global_models(train_df, expected_features, model_name_list, random_state=42, target_col='매출수량'):
    zoo = make_model_zoo(random_state=random_state)
    from collections import OrderedDict
    trained = OrderedDict()
    X = (train_df.reindex(columns=expected_features, fill_value=0.0)
                 .replace([np.inf, -np.inf], np.nan)
                 .fillna(0.0)
                 .astype(float))
    y = train_df[target_col].astype(float).values
    for name in model_name_list:
        if name not in zoo:
            print(f"[SKIP] {name}: not available")
            continue
        try:
            mdl = zoo[name][1]()
            mdl.fit(X, y)
            setattr(mdl, "_feature_list", expected_features[:])
            trained[name] = mdl
        except Exception as e:
            print(f"[SKIP] {name}: {e}")
    return trained

def _compute_step_ts_features(window_sales: pd.Series, need_cols: set[str]) -> dict:
    f = {}
    if window_sales is None or len(window_sales) == 0 or not need_cols:
        return f
    s = pd.to_numeric(window_sales, errors='coerce').fillna(0.0).astype(float)
    for k in (7, 14):
        key = f"lag_{k}"
        if key in need_cols:
            f[key] = float(s.iloc[-k]) if len(s) >= k else 0.0
    def _mean_last(w):
        tail = s.tail(w)
        return float(tail.mean()) if len(tail) > 0 else 0.0
    for w in (1, 3, 7, 14, 28):
        key = f"roll_mean_{w}"
        if key in need_cols: f[key] = _mean_last(w)
    def _std_last(w):
        tail = s.tail(w)
        return float(tail.std(ddof=1)) if len(tail) > 1 else 0.0
    for w in (3, 7, 14, 28):
        key = f"roll_std_{w}"
        if key in need_cols: f[key] = _std_last(w)
    if "nz_ratio_28" in need_cols:
        tail = s.tail(28)
        f["nz_ratio_28"] = float((tail > 0).mean()) if len(tail) > 0 else 0.0
    if "streak_pos" in need_cols:
        cnt = 0
        for v in reversed(s.values):
            if v > 0: cnt += 1
            else: break
        f["streak_pos"] = float(cnt)
    return f

def auto_regressive_predict_global_ensemble(
    models_dict, weights_dict=None, blend="weighted_mean",
    test_input_df=None, test_pred_df=None,
    drop_cols=None, expected_features=None,
    scaler_cols=None, scaler=None,
    window_size=28, dyn_keep=None, target_col='매출수량'
):
    assert blend in ("weighted_mean", "mean", "median")
    model_names = list(models_dict.keys())
    models = [models_dict[n] for n in model_names]
    feats_list = [_model_feature_names(m, expected_features) for m in models]
    if blend == "weighted_mean":
        if (weights_dict is None) or (len(weights_dict)==0):
            w = np.ones(len(models), dtype=float)
        else:
            w = np.array([weights_dict.get(n, 0.0) for n in model_names], dtype=float)
        w = np.clip(w, 0, None)
        w = w / (w.sum() if w.sum() > 0 else 1.0)
    else:
        w = np.ones(len(models), dtype=float) / max(1, len(models))
    dyn_keep = set(dyn_keep) if dyn_keep is not None else set(scaler_cols or [])
    dyn_keep &= set(expected_features or [])
    preds_all, nan_cnt, nan_examples = [], 0, []
    for (test_id, key), pred_group in test_pred_df.groupby(['test_id','영업장명_메뉴명']):
        cond = (test_input_df['test_id']==test_id) & (test_input_df['영업장명_메뉴명']==key)
        window = test_input_df[cond].sort_values('영업일자').copy()
        if '매출수량' not in window.columns:
            window['매출수량'] = 0.0
        rows = []
        for _, row in pred_group.sort_values('영업일자').iterrows():
            if dyn_keep:
                step_vals = _compute_step_ts_features(window[target_col], dyn_keep)
                if step_vals:
                    row.loc[list(step_vals.keys())] = list(step_vals.values())
            if scaler_cols and scaler is not None:
                vec = np.array([row.get(c, 0.0) for c in scaler_cols], dtype=float).reshape(1, -1)
                vec = np.nan_to_num(vec, nan=0.0, posinf=0.0, neginf=0.0)
                vec_scaled = scaler.transform(vec)[0]
                for c, v in zip(scaler_cols, vec_scaled):
                    row[c] = v
            m_preds = []
            for mdl, feats in zip(models, feats_list):
                xi = (row.drop(labels=(drop_cols or []) + [target_col], errors='ignore')
                        .reindex(feats, fill_value=0.0)
                        .astype(float)
                        .values.reshape(1,-1))
                try:
                    m_preds.append(float(mdl.predict(xi)[0]))
                except Exception:
                    m_preds.append(np.nan)
            m_preds = np.array(m_preds, dtype=float)
            if blend == "median":
                pred_raw = np.nanmedian(m_preds)
            elif blend == "mean":
                pred_raw = np.nanmean(m_preds)
            else:
                mask = np.isfinite(m_preds)
                if mask.any():
                    ww = w.copy(); ww[~mask] = 0.0
                    s = ww.sum()
                    pred_raw = float(np.dot(ww/s if s>0 else ww, np.nan_to_num(m_preds, nan=0.0))) if s>0 else float(np.nanmean(m_preds))
                else:
                    nan_cnt += 1
                    if len(nan_examples) < 5:
                        nan_examples.append({'test_id': test_id, '영업장명_메뉴명': key, '영업일자': row['영업일자']})
                    pred_raw = 0.0
            pred = np.clip(np.nan_to_num(pred_raw, nan=0.0), 0, None)
            rows.append({'test_id': test_id, '영업일자': row['영업일자'], '영업장명_메뉴명': key, '예측값': pred})
            add_row = row.copy(); add_row[target_col] = pred
            window = pd.concat([window, add_row.to_frame().T], ignore_index=True).sort_values('영업일자').iloc[-window_size:]
        if rows:
            preds_all.append(pd.DataFrame(rows))
    if nan_cnt>0 and nan_examples:
        print(f"[진단] (앙상블) raw NaN 예측 건수: {nan_cnt}")
        print("[예시] NaN 발생 샘플:", *nan_examples[:5], sep="\n")
    return pd.concat(preds_all, ignore_index=True) if preds_all else \
           pd.DataFrame(columns=['test_id','영업일자','영업장명_메뉴명','예측값'])

# -------------------------
# 8) (추가) AR-CV 수집 + SMAPE 출력
# -------------------------
def _safe_cutoffs(train_df: pd.DataFrame, n_folds: int, horizon: int, date_col: str = "영업일자"):
    days = np.array(sorted(pd.to_datetime(train_df[date_col].unique())))
    need = n_folds*horizon + 1
    if len(days) < need:
        step = max(1, len(days)//(n_folds+1))
        base = len(days) - horizon - 1
        idxs = list(range(base - (n_folds-1)*step, base+1, step))
        idxs = [i for i in idxs if 0 <= i < len(days)-1]
        return [days[i] for i in idxs]
    return list(days[-need:-1:horizon])

def run_time_series_cv_ar_collect(
    train_df,
    features,
    target_col="매출수량",
    n_folds=3,
    horizon=7,
    dyn_keep=None,
    scaler=None,
    scaler_cols=None,
    drop_cols=None,
    window_size=28,
    model_list=None
):
    train_df = train_df.sort_values("영업일자").copy()
    cutoffs = _safe_cutoffs(train_df, n_folds=n_folds, horizon=horizon, date_col="영업일자")
    if len(cutoffs) == 0:
        print("[경고] 유효한 CV 컷오프가 없습니다.")
        empty_oof = train_df[['업장명','영업일자', target_col]].copy()
        return empty_oof, pd.DataFrame(columns=["RMSE","MAE","SMAPE"])

    zoo_all = make_model_zoo()
    model_list = list(zoo_all.keys()) if model_list is None else model_list
    zoo = {k: zoo_all[k] for k in model_list if k in zoo_all}

    oof = train_df[['업장명','영업일자', target_col]].copy()
    for name in zoo.keys():
        oof[f'pred__{name}'] = np.nan

    model_scores = []

    for cutoff in cutoffs:
        fold_train = train_df[train_df['영업일자'] <= cutoff].copy()
        fold_valid = train_df[(train_df['영업일자'] > cutoff) &
                              (train_df['영업일자'] <= cutoff + pd.Timedelta(days=horizon))].copy()
        if fold_train.empty or fold_valid.empty:
            continue

        base_windows = {}
        for key, g in fold_train.groupby('영업장명_메뉴명'):
            s = pd.to_numeric(g.sort_values('영업일자')[target_col], errors='coerce').fillna(0.0)
            base_windows[key] = s

        X_train = (fold_train[features]
                   .replace([np.inf, -np.inf], np.nan)
                   .fillna(0.0)
                   .astype(float))
        y_train = fold_train[target_col].astype(float)

        for name,(_,builder) in zoo.items():
            try:
                mdl = builder()
                mdl.fit(X_train, y_train)
                preds_idx, preds_val = [], []
                window_by_key = {k: v.copy() for k, v in base_windows.items()}
                for day in sorted(fold_valid['영업일자'].unique()):
                    day_rows = fold_valid[fold_valid['영업일자']==day].copy()
                    for idx, row in day_rows.iterrows():
                        key = row['영업장명_메뉴명']
                        s = window_by_key.get(key, pd.Series(dtype=float))
                        if dyn_keep:
                            step_vals = _compute_step_ts_features(s, set(dyn_keep))
                            if step_vals:
                                for k,v in step_vals.items(): row[k] = v
                        if scaler is not None and scaler_cols:
                            vec = np.array([row.get(c, 0.0) for c in scaler_cols], dtype=float).reshape(1, -1)
                            vec = np.nan_to_num(vec, nan=0.0, posinf=0.0, neginf=0.0)
                            scaled = scaler.transform(vec)[0]
                            for c,v in zip(scaler_cols, scaled):
                                row[c] = float(v)
                        xi = (row.drop(labels=(drop_cols or [])+[target_col], errors='ignore')
                                .reindex(features, fill_value=0.0)
                                .astype(float)
                                .values.reshape(1,-1))
                        pred_raw = float(mdl.predict(xi)[0])
                        # 점수는 제출 정책과 동일하게 1min 클리핑 고려 시:
                        pred_score = 1.0 if (not np.isfinite(pred_raw) or pred_raw <= 1.0) else pred_raw
                        preds_idx.append(idx); preds_val.append(pred_score)
                        pred_for_window = 0.0 if (not np.isfinite(pred_raw) or pred_raw < 0.0) else pred_raw
                        s_new = pd.concat([s, pd.Series([pred_for_window])], ignore_index=True)
                        window_by_key[key] = s_new.iloc[-window_size:]
                if preds_idx:
                    oof.loc[preds_idx, f'pred__{name}'] = preds_val
                    tmp = fold_valid[[target_col,'업장명']].copy()
                    tmp['예측값'] = pd.Series(preds_val, index=preds_idx)
                    tmp = tmp.dropna()
                    if len(tmp) > 0:
                        rmse = mean_squared_error(tmp[target_col], tmp['예측값'], squared=False)
                        mae  = mean_absolute_error(tmp[target_col], tmp['예측값'])
                        s    = smape_group_macro(tmp)
                        model_scores.append((name, rmse, mae, s))
            except Exception as e:
                print(f"[SKIP fold] {name}: {e}")

    res_df = pd.DataFrame(model_scores, columns=["Model","RMSE","MAE","SMAPE"])
    model_cv = res_df.groupby("Model", as_index=True).mean().sort_values("SMAPE") if len(res_df) else \
               pd.DataFrame(columns=["RMSE","MAE","SMAPE"])
    print("\n[단일 모델 AR-CV 결과] (SMAPE 오름차순)")
    display(model_cv)
    return oof, model_cv

def evaluate_ensembles_on_oof(oof_df, weights_dict: dict, target_col='매출수량'):
    cols, weights = [], []
    for m, w in weights_dict.items():
        col = f'pred__{m}'
        if col in oof_df.columns and w>0:
            cols.append(col); weights.append(float(w))
    if not cols:
        print("[앙상블 CV] 사용할 예측 열이 없습니다.")
        return np.nan
    W = np.array(weights, dtype=float)
    W = W / W.sum() if W.sum()>0 else np.ones(len(W))/len(W)
    P = oof_df[cols].values
    mask = ~np.isnan(P)
    denom = (mask * W).sum(axis=1, keepdims=True)
    denom[denom==0] = np.nan
    blend = (np.nan_to_num(P) * W).sum(axis=1, keepdims=True) / denom
    blend = blend.ravel()
    tmp = pd.DataFrame({'업장명': oof_df['업장명'], target_col: oof_df[target_col], '예측값': blend}).dropna()
    s = smape_group_macro(tmp)
    print(f"[앙상블 CV SMAPE] {s:.6f}  (가중치: {weights_dict})")
    return s

[INFO] 최종 피처 개수: 30
['wd_mean_local_28', 'lag_14', 'lag_7', 'nz_ratio_28', 'roll_mean_1', 'roll_mean_28', 'roll_mean_3', 'roll_std_28', 'roll_std_3', 'roll_std_7', 'streak_pos', '일', '요일', '주차', '계절', '공휴일여부', '월_sin', '월_cos', '요일_sin', '요일_cos', 'is_weekend', '전일_공휴일', '익일_공휴일', '휴일전날', '휴일다음날', '연휴중여부', 'rfm_recency_train', 'rfm_freq_train', 'rfm_meanlead_train', 'rfm_avgqty_train']


In [8]:
# ============================================================
# 10) 고정 앙상블: 전체 학습 → 7일 AR 예측 → 제출 파일 생성
# ============================================================

# 1) 사용할 모델과 가중치 (설치된 모델만 자동 사용)
fixed_models  = ["XGB_Poisson","LGBM_Tweedie","Elastic"]
fixed_weights = {"XGB_Poisson":0.5, "LGBM_Tweedie":0.45, "Elastic":0.05}

# 2) 전체 train으로 모델 학습
trained = train_global_models(
    train_df=train,
    expected_features=expected_features,
    model_name_list=fixed_models,
    random_state=42,
    target_col=TARGET
)

if len(trained) == 0:
    raise RuntimeError("사용 가능한 모델이 없습니다. XGBoost/LightGBM 설치 여부를 확인하세요.")

# 3) 7일 자동회귀 예측 (고정 앙상블)
preds_long = auto_regressive_predict_global_ensemble(
    models_dict=trained,
    weights_dict=fixed_weights,         # 설치/학습된 모델만 내부에서 자동 정규화
    blend="weighted_mean",
    test_input_df=test_input_df,        # 히스토리
    test_pred_df=test_pred_df,          # 예측할 미래 날짜 표
    drop_cols=base_drop_cols,
    expected_features=expected_features,
    scaler_cols=scaler_cols,
    scaler=scaler,
    window_size=28,
    dyn_keep=dyn_keep,
    target_col=TARGET
)

# 4) 제출 포맷 복원 (sample_submission과 동일 인덱스/컬럼)
#    - sub_long에는 이미 test_id, offset, (실제)영업일자가 들어 있음
sub_for_join = sub_long[['test_id','offset','영업장명_메뉴명','영업일자']].copy()
sub_for_join['제출키'] = sub_for_join['test_id'] + '+' + sub_for_join['offset'].astype(int).astype(str) + '일'

merge_df = sub_for_join.merge(
    preds_long[['test_id','영업장명_메뉴명','영업일자','예측값']],
    on=['test_id','영업장명_메뉴명','영업일자'],
    how='left'
)

# 결측 방어 및 음수 방지
merge_df['예측값'] = pd.to_numeric(merge_df['예측값'], errors='coerce').fillna(0.0)
merge_df['예측값'] = merge_df['예측값'].clip(lower=0)

# 5) Wide 피벗 → sample_submission 스켈레톤에 채워 넣기
pred_wide = merge_df.pivot(index='제출키', columns='영업장명_메뉴명', values='예측값')

# sample_submission의 행/열 정렬을 그대로 유지
out_df = submission_df.set_index('영업일자').copy()
# 교집합 위치에 값 반영
out_df.update(pred_wide)

# # 6) 저장 (UTF-8-sig 요구사항)
SUBMIT_PATH = "submission_noweather.csv"
out_df.reset_index().to_csv(SUBMIT_PATH, index=False, encoding="utf-8-sig")
print(f"[OK] 저장 완료: {SUBMIT_PATH}  shape={out_df.shape}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003771 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3344
[LightGBM] [Info] Number of data points in the train set: 88844, number of used features: 30
[LightGBM] [Info] Start training from score 2.482751
[OK] 저장 완료: submission_noweather.csv  shape=(70, 167)


# 두 모델 결과 합치기

In [10]:
import pandas as pd
import numpy as np

# Load dataframes for temp_1 and temp_2
try:
    temp_1 = pd.read_csv('submission_noweather.csv')
    temp_2 = pd.read_csv('../data/submission.csv')
    
    # Melt each dataframe to a long format
    melted_1 = temp_1.melt(id_vars='영업일자', var_name='영업장명_메뉴명', value_name='pred_1')
    melted_2 = temp_2.melt(id_vars='영업일자', var_name='영업장명_메뉴명', value_name='pred_2')
    
    # Merge the two dataframes
    merged_preds = pd.merge(melted_1, melted_2, on=['영업일자', '영업장명_메뉴명'], how='inner')
    
    # Calculate the simple average of the two predictions
    merged_preds['final_pred'] = (merged_preds['pred_1'] + merged_preds['pred_2']) / 2
    
    # Clip negative values to 1
    merged_preds['final_pred'] = merged_preds['final_pred'].clip(lower=1)
    
    # Pivot the dataframe back to the final submission format
    final_submission = merged_preds.pivot(
        index='영업일자',
        columns='영업장명_메뉴명',
        values='final_pred'
    ).reset_index()
    
    # Reindex columns to match the sample submission if available
    try:
        sample_sub = pd.read_csv('data/sample_submission.csv')
        sample_cols = sample_sub.columns.tolist()
        final_submission = final_submission.reindex(columns=sample_cols, fill_value=1)
    except FileNotFoundError:
        print("sample_submission.csv 파일을 찾을 수 없어 컬럼 순서 정렬을 건너뜁니다.")
    
    # Save the final dataframe to a CSV file
    final_submission.to_csv('2_deep_average_.csv', index=False)
    
    print("두 개의 제출 파일의 예측값을 단순 평균한 새로운 제출 파일(2_high_average.csv)이 생성되었습니다.")

except FileNotFoundError as e:
    print(f"Error: {e}. 하나 이상의 입력 파일을 찾을 수 없습니다. 파일 경로를 확인해 주세요.")

sample_submission.csv 파일을 찾을 수 없어 컬럼 순서 정렬을 건너뜁니다.
두 개의 제출 파일의 예측값을 단순 평균한 새로운 제출 파일(2_high_average.csv)이 생성되었습니다.
